# 追蹤+結合滑台(3frame偵測回報，9frame驅動滑台)

In [ ]:
import torch
import cv2
import serial
import time
import numpy as np
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sys
sys.path.append('C:/Users/user/Yolov7/yolov7')  # 確保這個路徑是正確的
from models.yolo import Model  # 導入YOLOv7的Model類
import torchvision.transforms as transforms
from utils.general import non_max_suppression, scale_coords

# 自定義的繪製邊界框和標籤的函數
def draw_boxes(frame, boxes, labels, line_width, color):
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, line_width)
        cv2.putText(frame, label, ((x1 + x2) // 2, (y1 + y2) // 2), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

         
        
model_config = 'C:/Users/user/Yolov7/yolov7/cfg/training/yolov7_custom.yaml'
model_weights = 'C:/Users/user/Yolov7/yolov7/runs/train/exp30/weights/best.pt'

class_names = ["BMW", "Fiat", "Ford", "Honda", "Long", "Mazda", "Mercedes", "Nissan", "Toyota", "Volkswagen"]
frame_counter = 0

# 用於跟踪未檢測到高置信度物體的次數
no_detection_counter = 0

# 載入模型配置
model = Model(model_config)

# 載入預訓練的權重
ckpt = torch.load(model_weights, map_location='cpu')
model.load_state_dict(ckpt['model'].state_dict())

# 定義神經網路結構
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 2)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        return self.layer4(x)

# 載入模型參數
model_nn = NeuralNetwork(4)  # 使用不同名稱避免混淆
model_nn.load_state_dict(torch.load('trained_model20231001.pth'))
model_nn.eval()

# 載入標準化器
data_path = "C:/Users/user/點位記錄整理.xlsx"
df = pd.read_excel(data_path)
features = df[['px', 'py', 'sx', 'sy']].values
scaler = StandardScaler().fit(features)

class StageCommander:
    def __init__(self, port='COM3', baudrate=38400):
        self.ser = serial.Serial(port, baudrate)
        self.ser.timeout = 0.05
        self.current_position = (0, 0)

    def send_command(self, command):
        self.ser.write((command + '\r').encode())
        time.sleep(0.01)
        return self.ser.readline().decode().strip()

    def set_pulse(self, axis, pulse):
        command = f"AXI{axis}:PULSe {pulse}"
        response = self.send_command(command)

    def drive_to_relative_position(self, x_diff, y_diff):
        x_direction = 'X+' if x_diff > 0 else 'X-'
        y_direction = 'Y+' if y_diff > 0 else 'Y-'

        self.set_pulse('X', abs(x_diff))
        self.set_pulse('Y', abs(y_diff))

        position = f"{x_direction}{y_direction}"
        response = self.send_command(f"GOLI {position}")

        self.current_position = (self.current_position[0] + x_diff, self.current_position[1] + y_diff)
        
        # 顯示滑台當前位置到小數點後第三位
        print(f"滑台當前位置: ({self.current_position[0]:.3f}, {self.current_position[1]:.3f})")

    def close(self):
        self.ser.close()

# 初始化滑台控制器
commander = StageCommander('COM3', 38400)
commander.current_position = (0, 0)  # 設置滑台初始位置為 (0,0)

# 初始化攝像頭
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("無法打開攝像頭")
    exit()

last_position = None
frame_counter = 0
over_limit_counter = 0  # 追踪超過限制的次數

while True:
    # 捕獲即時影像
    ret, frame = cap.read()
    if not ret:
        print("無法讀取攝像頭影像")
        break       

    frame_counter += 1

    # 每3個 frame 進行一次檢測
    if frame_counter % 3 == 0:
        # 將 BGR 影像轉為 RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((640, 480)),  # 假設模型需要的輸入尺寸
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        frame_transformed = transform(frame_rgb)
        frame_batch = frame_transformed.unsqueeze(0)

        # 進行檢測
        model.eval()
        with torch.no_grad():
            results = model(frame_batch)

            # 處理模型輸出
            if isinstance(results, tuple):
                predictions = results[0]
            else:
                predictions = results

            # 設置閾值
            conf_thres = 0.6
            iou_thres = 0.45
            classes = None
            agnostic_nms = False

            # 後處理
            pred = non_max_suppression(predictions, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)

            detected = False
            
            # 顯示檢測結果
            for i, det in enumerate(pred):
                if len(det):
                    det[:, :4] = scale_coords(frame_batch.shape[2:], det[:, :4], frame.shape).round()

                    boxes = []
                    labels = []
                    for *xyxy, conf, cls in det:
                        if conf >= conf_thres:
                            detected = True
                            x1, y1, x2, y2 = map(int, xyxy)
                            label = f'{class_names[int(cls)]}'
                            
                            A = 250
                            B = 2
                            if  480 > y1 > A:
                                y1 = (y1 - A) / B + A
                                if 480 > y2 > A:
                                    y2 = (y2 - A) / B + A
                                else:
                                    y2 = y2
                            elif 0 < y1 < A:
                                y1 = A - ((A - y1) / B)
                                if 480 > y2 > A:
                                    y2 = (y2 - A) / B + A
                                elif 0 < y2 < A:
                                    y2 = A - ((A - y2) / B)
                                elif y2 == 480:
                                    y2 = 432
                                elif y2 == 0:
                                    y2 = 125
                                else:
                                    y2 = y2
                            elif y1 == 480:
                                y1 = 432
                                y2 = 480
                                
                            elif y1 == 0:
                                y1 = 50
                                if 480 > y2 > A:
                                    y2 = (y2 - A) / B + A
                                elif 0 < y2 < A:
                                    y2 = A - ((A - y2) / B)
                                elif y2 == 480:
                                    y2 = 432
                                elif y2 == 0:
                                    y2 = 75
                                else:
                                    y2 = y2
                            else:
                                y1 = A
                                if 480 > y2 > A:
                                    y2 = (y2 - A) / B + A
                                else:
                                    y2 = 432
                            
                            boxes.append([x1, y1, x2, y2])
                            labels.append(label)

                            # 計算物體中心點座標
                            centerX = int((x1 + x2) // 2)
                            centerY = int((y1 + y2) // 2)

                            # 打印資訊
                            print(f'Frame {frame_counter}: 檢測到的物體: {label}, 置信度: {conf:.2f}, 中心點座標: ({centerX}, {centerY})')

                    if detected:                       
                        draw_boxes(frame, boxes, labels, 5, (0, 255, 0))  # 綠色邊界框和標籤

            if not detected:
                no_detection_counter += 1
                print(f"Frame {frame_counter}: 沒有檢測到置信度大於0.6的物體。")
                if no_detection_counter >= 1000:
                    print("連續二十次沒有追踪到置信度大於0.6的物體，中止程式。")
                    break
            else:
                no_detection_counter = 0

        if detected and frame_counter % 9 == 0:
            # 使用檢測到的物體中心點進行XY滑台控制
            px, py = centerX, centerY  # CCD所看到的中心坐標
            sx, sy = commander.current_position  # 滑台當前的位置

            # 標准化输入數據
            sample_data = scaler.transform([[px, py, sx, sy]])
            sample_data = torch.tensor(sample_data, dtype=torch.float32)

            # 使用模型進行預測
            predicted_output = model_nn(sample_data)
            predicted_x, predicted_y = predicted_output[0].detach().numpy()

            # 調整預測移動量以符合限制
            if sx + predicted_x > 48:
                predicted_x = 48 - sx
            elif sx + predicted_x < -48:
                predicted_x = -48 - sx

            if sy + predicted_y > 48:
                predicted_y = 48 - sy
            elif sy + predicted_y < -48:
                predicted_y = -48 - sy

            # 控制XY滑台
            commander.drive_to_relative_position(predicted_x, predicted_y)

            # 更新滑台當前位置
            new_sx = sx + predicted_x
            new_sy = sy + predicted_y
            commander.current_position = (new_sx, new_sy)

            # 檢查是否連續超過限制
            if abs(new_sx) > 48 or abs(new_sy) > 48:
                over_limit_counter += 1
                if over_limit_counter >= 5:
                    print("連續5次超過位置限制，終止程式")
                    break
            else:
                over_limit_counter = 0
                
    cv2.imshow('YOLOv7 Object Detection', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
commander.close()

# 回歸原點

In [ ]:
import serial
import time

class StageCommander:
    def __init__(self, port='COM3', baudrate=38400):
        self.ser = serial.Serial(port, baudrate)
        self.ser.timeout = 0.05
        # 初始化座標為(0,0)
        self.x_position = 0
        self.y_position = 0
    
    def send_command(self, command):
        self.ser.write((command + '\r').encode())
        time.sleep(0.01)
        return self.ser.readline().decode().strip()
    
    def set_pulse(self, axis, pulse):
        command = f"AXI{axis}:PULSe {pulse}"
        response = self.send_command(command)
    
    def drive_to_relative_position(self, position, wait_time=0):
        start_time = time.time()
        command = f"GOLI {position}"
        response = self.send_command(command)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Response: {response}, Elapsed time: {elapsed_time} seconds")
        time.sleep(wait_time)
        
        # 更新座標
        if 'X+' in position:
            self.x_position += abs(x_pulse)
        elif 'X-' in position:
            self.x_position -= abs(x_pulse)
        if 'Y+' in position:
            self.y_position += abs(y_pulse)
        elif 'Y-' in position:
            self.y_position -= abs(y_pulse)
    
    def get_position(self):
        return self.x_position, self.y_position
    
    def close(self):
        self.ser.close()

# 使用範例:
commander = StageCommander('COM3', 38400)

while True:
    xy_pulse = input("請同時輸入X軸和Y軸的移動pulse (例如: 1,-1 或 -1,1 或 q結束): ")
    if xy_pulse.lower() == 'q':
        break

    x_pulse, y_pulse = map(float, xy_pulse.split(','))
    x_direction = 'X+' if x_pulse > 0 else 'X-'
    y_direction = 'Y+' if y_pulse > 0 else 'Y-'

    commander.set_pulse('X', abs(x_pulse))
    commander.set_pulse('Y', abs(y_pulse))

    position = f"{x_direction}{y_direction}"
    commander.drive_to_relative_position(position, wait_time=1.8)

    # 顯示滑台的座標
    x_position, y_position = commander.get_position()
    print(f"滑台目前的座標: X={x_position}, Y={y_position}")

commander.close()